## Installation and import

In [ ]:
import os

# dataset
dataset_name="BERT_HERV_Multi"
#### COMET_API_KEY
os.environ['COMET_API_KEY'] = "4H9lGndNp1kftyPBrDQw5Mn3F"
api_key="4H9lGndNp1kftyPBrDQw5Mn3F"
project_name="dna-finetuning-"+str(dataset_name)
workspace="george-du"

os.environ['COMET_PROJECT_NAME'] = project_name
os.environ['COMET_WORKSPACE'] = workspace

In [ ]:
import comet_ml
from comet_ml import Experiment
import torch
from torch.optim import AdamW
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding,TrainingArguments, Trainer, EarlyStoppingCallback
from transformers.integrations import CometCallback
from itertools import product
import os
from collections import Counter
import pandas as pd
import numpy as np
from random import random, randrange
import random
from datasets import Dataset, DatasetDict, load_metric
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score,matthews_corrcoef
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
import numpy as np

torch.cuda.empty_cache()
torch.__version__


In [ ]:
#### 返回当前可用的内存
import GPUtil

def get_available_gpu_memory():
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        if gpu.id == torch.cuda.current_device():
            return gpu.memoryFree  # 返回可用内存，单位是MB

## dataset input

In [ ]:
file="/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa"
#file="/home/duduo/Project/DNA-Pretraining/Level1/002.Model_Classification/Dataset_HERV/data_all_model_HERV-Classification_Need_NEW-test.fa"
output_dir="/home/u20111010010/Project/DNA-Pretraining/Level1/002.Model_Classification/Dataset_HERV"

#类别与编号关系
id2label={"0":"Non-HERV_Coding","1":"HERV_Coding","2":"Non-HERV_Non-Coding","3":"HERV_Non-Coding"}
label2id={"Non-HERV_Coding":"0","HERV_Coding":"1","Non-HERV_Non-Coding":"2","HERV_Non-Coding":"3"}

## Initialization parameters

In [ ]:
### Parameters
MODEL_NAME = "/home/u20111010010/Project/DNA-Pretraining/Level1/002.Model_Classification/Pre-train_Model/DNA_bert_6"       #
TOKENIZER_NAME = "/home/u20111010010/Project/DNA-Pretraining/Level1/002.Model_Classification/Pre-train_Model/DNA_bert_6"   #

K = 6
STRIDE = 1

# train
BATCH_SIZE = 10  ### 10
ACCUMULATION = 4   #2到4
LEARNING_RATE =2e-5   #LEARNING_RATE = 1e-6，学习率由8e-5至2e-5
EPOCHS = 50
RUNS = 3
warmup_ratio = 0.1 #5 epochs (for 100 epochs total train)，学习率的调整超参数;0.05到0.1

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_NAME, 
    model_max_length=512,
)

if(K is not None and K>6):
    alphabet = ('A', 'C', 'T', 'G')
    vocab = list(map(''.join, product(alphabet, repeat=K)))
    tokenizer.add_tokens(vocab)
#如果一个序列的长度为L，K-mer的长度为k，那么该序列最终可以得到L-k+1个子序列

In [ ]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

####### 截取序列中前300个tokens+后212个tokens
if(K is not None):
    def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])[:300] + kmers(x["seq"])[-212:]), truncation=True)
else:
    def tok_func(x): return tokenizer(x["seq"][:300] + x["seq"][-212:], truncation=True)

# example
example = tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})
print(example)
tokenizer.decode(example['input_ids'])

## evaluate data and Model(Fine-tuning)

In [ ]:
def compute_metrics_multi_class(eval_preds):
    logits, labels = eval_preds
    prediction_scores = logits  # Assuming the logits are the prediction scores
    predictions = np.argmax(prediction_scores, axis=-1)

    # For multi-class case, AUC and PR AUC should be calculated in one-vs-all manner
    lb = LabelBinarizer()
    lb.fit(labels)
    labels_binarized = lb.transform(labels)
    prediction_scores_binarized = lb.transform(predictions)
    
    # 计算各种评价指标
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    roc_auc_macro = roc_auc_score(labels_binarized, prediction_scores_binarized, average='macro', multi_class='ovr')
    roc_auc_weighted = roc_auc_score(labels_binarized, prediction_scores_binarized, average='weighted', multi_class='ovr')
    pr_auc = average_precision_score(labels_binarized, prediction_scores_binarized, average='weighted')
    mcc = matthews_corrcoef(labels, predictions)
    
    # 返回评价指标
    return {
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall,
        'precision': precision,
        'roc_auc_macro': roc_auc_macro,
        'roc_auc_weighted': roc_auc_weighted,
        'pr_auc': pr_auc,
        'mcc': mcc  # 返回MCC指标
    }

In [ ]:
def get_log_from_history(history, dataset_name):
    eval_dicts = [x for x in history if 'eval_loss' in x]
    test_dicts = [x for x in history if 'test_loss' in x]
    test_log = test_dicts[0]
    test_acc = test_log['test_accuracy']
    test_f1 = test_log['test_f1']
    test_loss = test_log['test_loss']
    test_precision = test_log['test_precision']
    test_recall = test_log['test_recall']
    test_auroc_macro = test_log['test_roc_auc_macro']
    test_auroc_weighted = test_log['test_roc_auc_weighted']
    test_pr_auc = test_log['test_pr_auc']
    test_mcc=test_log['test_mcc']
    
    train_runtime = max([x['train_runtime'] for x in history if 'train_runtime' in x])  
    min_loss_dict = min(eval_dicts, key=lambda x: x['eval_loss'])
    min_loss_epoch = min_loss_dict['epoch']
    # max_f1_dict = max(eval_dicts, key=lambda x: x['eval_f1'])
    # max_acc_dict = max(eval_dicts, key=lambda x: x['eval_accuracy'])
    row = {
        'dataset':dataset_name,
        'test_acc':test_acc,
        'test_f1':test_f1,
        'test_loss':test_loss,
        'test_precision':test_precision,
        'test_recall':test_recall,
        'test_auroc_macro':test_auroc_macro,
        'test_auroc_weighted':test_auroc_weighted,
        'test_pr_auc':test_pr_auc,
        'train_runtime':train_runtime,
        'min_valid_loss_epoch':min_loss_epoch,
        'min_valid_loss_log':min_loss_dict,
        'test_mcc':test_mcc
        # 'max_valid_f1_log':max_f1_dict,
        # 'max_valid_acc_log':max_acc_dict,
    }
    return row

In [ ]:
df1=pd.read_csv(file,sep="\t",header=None).rename(columns = {0: "dset", 1: "multi",2:"binary", 3: "seq",4:"Type",5: "detail"})
df = df1.loc[:, ['dset', 'multi','seq']]
ds = Dataset.from_pandas(df)
print(df.head())
print(ds[0])

In [ ]:
labels=set(df['multi'])
tok_ds = ds.map(tok_func, batched=False)

tok_ds = tok_ds.rename_columns({'multi':'labels'})
dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

train_valid_split = dds['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)
dds['train']=train_valid_split['train']
dds['eval']=train_valid_split['test']

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #数据调度
compute_metrics = compute_metrics_multi_class

In [ ]:
### 自定义权重函数

def getweights(dataset):
    label_counts = Counter(dataset['labels'])
    total_count = len(dataset['labels'])
    weights_list = [total_count / label_counts.get(i, 1) for i in sorted(label_counts.keys())]
    weights_tensor = torch.tensor(weights_list, dtype=torch.float32)
    return weights_tensor

### 动态地添加getweights函数 
def add_getweights_to_dataset(dataset):
    dataset.getweights = lambda: getweights(dataset)
    return dataset

### 自定义Trainer函数
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super(CustomTrainer, self).__init__(*args, **kwargs)
        
        # 计算并存储 weights_tensor
        self.train_dataset = add_getweights_to_dataset(self.train_dataset)
        self.weights_tensor = self.train_dataset.getweights()
        
        rank = os.environ.get("LOCAL_RANK", 0)  # 获取每个进程在所在主机中的序号
        this_device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
        self.weights_tensor = self.weights_tensor.to(this_device)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # 注意这里使用已经计算好的 self.weights_tensor
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [ ]:
outputs = []

### 测试集真实结果
Test_results = pd.DataFrame(df[df['dset'] == 'test'])

for _ in range(RUNS):
    
    model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels),id2label=id2label,label2id=label2id)

    experiment = Experiment(api_key = api_key,project_name =project_name,workspace=workspace,display_summary_level=1)
    #experiment.set_name(str(dataset_name) + "_RUN" + str(_))
    
    if torch.cuda.device_count() > 0:model_cls.to('cuda')

    ### Hugging Face Transformers库模型默认会全参微调，可以将模型的顶层任务特定层解冻并用于微调，而底层的预训练层被冻结，而不会被微调
    AVAILABLE_GPU_MEMORY = get_available_gpu_memory()
    eval_batch_size = int(min(BATCH_SIZE * 2, AVAILABLE_GPU_MEMORY // (512 * 4 * 2)+1))  # 512是模型的最大序列长度，4是float32的字节数，2是安全系数
    
    args=TrainingArguments(output_dir=os.path.join(output_dir,"Model",str(dataset_name)),logging_dir=os.path.join(output_dir,"Model",str(dataset_name)),overwrite_output_dir=True, learning_rate=LEARNING_RATE, warmup_ratio=warmup_ratio, lr_scheduler_type='cosine',fp16=True,evaluation_strategy="epoch", 
        per_device_train_batch_size=BATCH_SIZE,per_device_eval_batch_size=eval_batch_size,gradient_accumulation_steps=ACCUMULATION,num_train_epochs=EPOCHS, 
        weight_decay=0.01,save_strategy='epoch', report_to='none',metric_for_best_model="eval_loss",load_best_model_at_end=True,seed=randrange(1,10001),greater_is_better=False
    )#linear

    #early stopping 5 epochs
    callbacks= [EarlyStoppingCallback(early_stopping_patience=10, early_stopping_threshold=0.05),CometCallback()]
    
    trainer = CustomTrainer(model_cls, args, train_dataset=dds['train'],eval_dataset=dds['eval'],
        tokenizer=tokenizer, compute_metrics=compute_metrics,callbacks=callbacks,data_collator=data_collator
    )
    trainer.train()
        
    ### 测试集进行评估
    trainer.evaluate(dds['test'], metric_key_prefix='test')
    training_log = get_log_from_history(trainer.state.log_history, dataset_name=dataset_name)
    outputs.append(training_log)
    
    ### 测试集预测结果保存
    predictions = trainer.predict(dds['test'])
    predicted_labels = np.argmax(predictions.predictions, axis=1)
    true_labels = predictions.label_ids
    
    confusion = confusion_matrix(true_labels, predicted_labels)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++ Confusion Matrix:Start "+"RUN"+str(_))
    print(confusion)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++ Confusion Matrix:End "+"RUN"+str(_))
    
    # 将这一次的预测标签添加到结果DataFrame中
    Test_results[f'predicted_labels{_}'] = predicted_labels.tolist()
   
    ### 模型保存
    trainer.save_model(os.path.join(output_dir,"Model",str(dataset_name)+"_RUN"+str(_)))
    experiment.log_confusion_matrix(labels=list(id2label.values()),
                                    matrix=confusion)
    experiment.end()

## Evaluate and present the results (Test;One by One)

In [ ]:
print(trainer.state.log_history)

In [ ]:
outputs_df = pd.DataFrame(outputs)
outputs_df.to_csv(os.path.join(output_dir,str(dataset_name)+"-Label_Times"+str(RUNS)+".csv"), index=True)
outputs_df

In [ ]:
### SEM (Standard Error of the Mean)
outputs_df2=outputs_df.groupby('dataset').agg({'test_acc' : ['mean', 'sem'], 'test_f1' : ['mean','sem'],'test_loss' : ['mean','sem'],'test_precision' : ['mean','sem'],'test_recall' : ['mean','sem'],'test_auroc_macro' : ['mean','sem'],'test_pr_auc' : ['mean','sem'],'test_mcc' : ['mean','sem'],'train_runtime': ['mean', 'sem']})
outputs_df2.to_csv(os.path.join(output_dir,str(dataset_name)+"-Label_Times"+str(RUNS)+"_Mean.csv"), index=False)
outputs_df2

## Evaluate and present the results (Test;Merge-more than twice)

In [ ]:
modes = Test_results[['predicted_labels0', 'predicted_labels1', 'predicted_labels2']].mode(axis=1)
modes_count = Test_results[['predicted_labels0', 'predicted_labels1', 'predicted_labels2']].apply(lambda x: x.value_counts().max(), axis=1)

# 创建新列，将众数放入新列
Test_results['predicted_labels_new'] = Test_results['predicted_labels0']

# 根据重复值最多的数值更新新列的值
Test_results.loc[modes_count > 1, 'predicted_labels_new'] = modes[modes_count > 1].iloc[:, 0]

In [ ]:
Test_results.to_csv(os.path.join(output_dir,str(dataset_name)+"-Label_Times"+str(RUNS)+"_prediction_results.csv"), index=False)
Test_results.head()

In [ ]:
def compute_metrics_multi_label(labels, predictions):
    # 计算各种评价指标
    
    # For multi-class case, AUC and PR AUC should be calculated in one-vs-all manner
    lb = LabelBinarizer()
    lb.fit(labels)
    labels_binarized = lb.transform(labels)
    prediction_scores_binarized = lb.transform(predictions)
    
    # 计算各种评价指标
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    roc_auc_macro = roc_auc_score(labels_binarized, prediction_scores_binarized, average='macro', multi_class='ovr')
    roc_auc_weighted = roc_auc_score(labels_binarized, prediction_scores_binarized, average='weighted', multi_class='ovr')
    pr_auc = average_precision_score(labels_binarized, prediction_scores_binarized, average='weighted')
    mcc = matthews_corrcoef(labels, predictions)
    
    # 返回评价指标
    return {
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall,
        'precision': precision,
        'roc_auc_macro': roc_auc_macro,
        'roc_auc_weighted': roc_auc_weighted,
        'pr_auc': pr_auc,
        'mcc': mcc,  # 返回MCC指标
        #'confusion':confusion
    }

In [ ]:
compute_metrics_multi_label(Test_results['multi'],Test_results['predicted_labels_new'])

In [ ]:
outputs_df3= pd.DataFrame.from_dict(compute_metrics_multi_label(Test_results['multi'],Test_results['predicted_labels_new']), orient='index', columns=['Value'])
outputs_df3.index.name = 'Metric'

outputs_df3.to_csv(os.path.join(output_dir,str(dataset_name)+"-Label_Times"+str(RUNS)+"_Vote.csv"), index=False)
outputs_df3

### 预测效果与序列长度变化

In [ ]:
filename="/home/u20111010010/Project/DNA-Pretraining/Level1/002.Model_Classification/Dataset_HERV/BERT_HERV_Multi-Label_Times3_prediction_results.csv"

In [ ]:
def compute_metric(labels, predictions, metric_name, lb):
    if metric_name == 'accuracy':
        return accuracy_score(labels, predictions)
    elif metric_name == 'f1':
        return f1_score(labels, predictions, average='weighted')
    elif metric_name == 'recall':
        return recall_score(labels, predictions, average='weighted')
    elif metric_name == 'precision':
        return precision_score(labels, predictions, average='weighted')
    elif metric_name == 'roc_auc_macro':
        lb.fit(labels)
        return roc_auc_score(lb.transform(labels), lb.transform(predictions), average='macro', multi_class='ovo')
    elif metric_name == 'roc_auc_weighted':
        lb.fit(labels)
        return roc_auc_score(lb.transform(labels), lb.transform(predictions), average='weighted', multi_class='ovo')
    elif metric_name == 'pr_auc':
        lb.fit(labels)
        return average_precision_score(lb.transform(labels), lb.transform(predictions), average='weighted')
    elif metric_name == 'mcc':
        return matthews_corrcoef(labels, predictions)
    else:
        raise ValueError(f"Unknown metric: {metric_name}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef, classification_report
from sklearn.preprocessing import LabelBinarizer


df = pd.read_csv(filename, sep=',')
df['Length'] = df['seq'].str.len()

# 对 'Length' 列进行对数变换
df['log_Length'] = np.log(df['Length'])

predicted_labels_columns = ['predicted_labels0', 'predicted_labels1', 'predicted_labels2', 'predicted_labels_new']
new_labels = ['RUN0', 'RUN1', 'RUN2', 'RUN_Vote']

# 将 log_Length 划分为 bins，计算每个 bin 的平均值
bins = np.linspace(df['log_Length'].min(), df['log_Length'].max(), 20)
df['adjusted_log_Length'] = df['log_Length'] + np.random.uniform(-0.1, 0.1, size=len(df))
bins = np.linspace(df['adjusted_log_Length'].min(), df['adjusted_log_Length'].max(), 20)
df['binned_log_Length'] = pd.cut(df['adjusted_log_Length'], bins=bins)

In [ ]:
def merge_bins_with_few_data_points(bin_counts, min_count=100):
    # 将区间转换为列表
    bins = list(bin_counts.index)
    counts = list(bin_counts.values)
    # 存储合并后的区间
    merged_bins = []
    while bins:
        current_bin = bins.pop()
        current_count = counts.pop()
        while current_count < min_count and bins:
            # 合并当前区间与前一个区间
            previous_bin = bins.pop()
            previous_count = counts.pop()
            current_bin = pd.Interval(previous_bin.left, current_bin.right)
            current_count += previous_count
        merged_bins.append(current_bin)
    return merged_bins[::-1]  # 反转列表，使区间按照从小到大的顺序排列

# 使用函数
bin_counts = df['binned_log_Length'].value_counts()
merged_intervals = merge_bins_with_few_data_points(bin_counts)

# 在df中添加新的列
df['binned_log_Length_New'] = pd.cut(df['log_Length'], bins=sorted(set([interval.left for interval in merged_intervals] + [merged_intervals[-1].right])))

In [ ]:
# 设置图像特征
plt.figure(figsize=(10,10))
#plt.figure()
plt.ylim(0, 1)

metrics_list = ['accuracy', 'f1', 'recall', 'precision', 'roc_auc_macro', 'roc_auc_weighted', 'pr_auc', 'mcc']
colors = plt.cm.viridis(np.linspace(0, 1, len(metrics_list)))
markers = ['o', 's', '^', 'D']

# 获取当前轴
ax1 = plt.gca()

# 密度图
def smooth(y, box_pts):
    """Applies a moving average to a signal."""
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

bin_counts = df['binned_log_Length_New'].value_counts().sort_index()
bin_densities = bin_counts / len(df)
bin_densities_smooth = smooth(bin_densities.values, 3)
ax1.plot(bin_densities.index.astype(str), bin_densities_smooth, color='grey', alpha=0.5, linewidth=2)

# 添加log(512)的竖线
ax1.axvline(x=np.log(512), color='red', linestyle='--', linewidth=2)
ax1.text(np.log(512)+0.5, 0.7, 'Max Tokens(512)', color='black', ha='left', va='center', rotation=0)

handles1 = []
handles2 = []

lb = LabelBinarizer()

for metric_index, metric_name in enumerate(metrics_list):
    for column_index, (column, new_label) in enumerate(zip(predicted_labels_columns, new_labels)):
        grouped = df.groupby('binned_log_Length_New').apply(
            lambda group: compute_metric(group['multi'], group[column], metric_name, lb)).reset_index(name=f'{metric_name}_{new_label}')
        
        ax1.plot(grouped.index, grouped[f'{metric_name}_{new_label}'],
                 color=colors[metric_index],
                 marker=markers[column_index],
                 linestyle='-',
                 label=f'{new_label}_{metric_name}')
        
        if metric_index == 0:
            handles1.append(mlines.Line2D([], [], color='black', marker=markers[column_index], linestyle='', label=new_label))
        
        if column_index == 0:
            handles2.append(mlines.Line2D([], [], color=colors[metric_index], label=metric_name))

# 设置标签和标题
ax1.set_xticklabels(bin_densities.index.astype(str), rotation=45)
ax1.set_xlabel('Log of Sequence Length')
ax1.set_ylabel('Metric Value')
ax1.set_title('HERV-Multi-Label(DNA_bert_6)')

# 创建合并的图例
handles = handles1 + handles2
labels = new_labels + metrics_list

# 位置 'lower right' 将图例放在图的右下角
plt.legend(handles, labels, loc='lower right', title='Runs & Metrics')

#plt.subplots_adjust(right=0.7)  # Adjust the right boundary of the plot window
plt.savefig(f'{filename}.pdf', bbox_inches='tight')
plt.savefig(f'{filename}.png', bbox_inches='tight')
#plt.show()

In [ ]:
#### 更新实际绘图
### 密度图放在最上边
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, average_precision_score, matthews_corrcoef
import matplotlib.lines as mlines

fontdict_bold = {'fontsize': 14}
ax1.set_ylabel('Rate',fontdict = fontdict_bold)

# 设置图像特征，展示在上边
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 8], hspace=0.005)  # 设置hspace=0确保两个子图之间无间隙
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1], sharex=ax1)

ax1.set_ylim(0.01, 0.3)  # 调整密度图的Y轴起始位点
ax2.set_ylim(0, 1)    # 调整指标图的Y轴终止位点
ax1.set_ylabel('Rate',fontdict = fontdict_bold)

# 隐藏密度图的X轴刻度和标签
ax1.set_xticks([])
ax1.xaxis.set_ticks_position('none')
ax1.xaxis.set_visible(False)

metrics_list = ['accuracy', 'f1', 'recall', 'precision', 'roc_auc_macro', 'roc_auc_weighted', 'pr_auc', 'mcc']
colors = plt.cm.viridis(np.linspace(0, 1, len(metrics_list)))
markers = ['o', 's', '^', 'D']

def smooth(y, box_pts):
    """Applies a moving average to a signal."""
    box = np.ones(box_pts) / box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

bin_counts = df['binned_log_Length_New'].value_counts().sort_index()
bin_densities = bin_counts / len(df)
bin_densities_smooth = smooth(bin_densities.values, 3)
ax1.plot(bin_densities.index.astype(str), bin_densities_smooth, color='grey', alpha=0.5, linewidth=2)
ax1.set_title('HERV-Multi-Label(DNA_bert_6)',fontdict = fontdict_bold)  # 将标题移到密度图上

ax2.axvline(x=np.log(512), color='red', linestyle='--', linewidth=2)
ax2.text(np.log(512) - 2.5, 0.7, 'Max Tokens(512)', color='black', ha='left', va='center', rotation=0)

handles1 = []
handles2 = []
lb = LabelBinarizer()

for metric_index, metric_name in enumerate(metrics_list):
    for column_index, (column, new_label) in enumerate(zip(predicted_labels_columns, new_labels)):
        grouped = df.groupby('binned_log_Length_New').apply(
            lambda group: compute_metric(group['multi'], group[column], metric_name, lb)).reset_index(name=f'{metric_name}_{new_label}')
        
        ax2.plot(grouped.index, grouped[f'{metric_name}_{new_label}'],
                 color=colors[metric_index],
                 marker=markers[column_index],
                 linestyle='-',
                 label=f'{new_label}_{metric_name}')
        
        if metric_index == 0:
            handles1.append(mlines.Line2D([], [], color='black', marker=markers[column_index], linestyle='', label=new_label))
        
        if column_index == 0:
            handles2.append(mlines.Line2D([], [], color=colors[metric_index], label=metric_name))

# 设置标签和标题
ax2.set_xticks(range(len(bin_densities.index)))
ax2.set_xticklabels(bin_densities.index.astype(str), rotation=45)
ax2.set_xlabel('Log-Scaled Binning of Sequence Length',fontdict = fontdict_bold)
ax2.set_ylabel('Metric Value',fontdict = fontdict_bold)

# 创建合并的图例
handles = handles1 + handles2
labels = new_labels + metrics_list

# 位置 'lower right' 将图例放在图的右下角
ax2.legend(handles, labels, loc='lower right', title='Runs & Metrics')
plt.tight_layout()

#plt.subplots_adjust(right=0.7)  # Adjust the right boundary of the plot window
plt.savefig(f'{filename}.pdf', bbox_inches='tight')
plt.savefig(f'{filename}.png', bbox_inches='tight',dpi=2000)